In [ ]:
# Imports 

# Pandas: Data manipulation and analysis
import pandas as pd

# NumPy: Scientific computing with arrays, array manipulation
import numpy as np

# Matplotlib: Plotting library
import matplotlib.pyplot as plt

# sys: Access system-specific parameters and functions
import sys

# os: Interact with the operating system
import os

# importlib: Dynamic module import
import importlib

# contextlib: Utilities for working with context managers
from contextlib import contextmanager

In [ ]:
# Context manager to temporarily add a directory to sys.path
# This ensures the directory is added to sys.path when entering the context
# and removed from sys.path when exiting the context
@contextmanager
def add_sys_path(path):
    """
    Temporarily adds the specified path to sys.path.

    Parameters:
    path (str): The directory path to add to sys.path.

    Yields:
    None
    """
    # Check if the specified path is already in sys.path
    if path not in sys.path:
        # If not, append the path to sys.path
        sys.path.append(path)
    try:
        # Yield control back to the calling context
        # Code within the 'with' block will execute at this point
        yield
    finally:
        # This block will always execute upon exiting the context
        # Check if the path is in sys.path
        if path in sys.path:
            # If it is, remove the path from sys.path
            sys.path.remove(path)

# Function to import and reload modules if necessary
def import_and_reload(module_name, path):
    """
    Imports and reloads the specified module from the given path.

    Parameters:
    module_name (str): The name of the module to import and reload.
    path (str): The directory path where the module is located.

    Returns:
    module: The imported and reloaded module.
    """
    # Use the add_sys_path context manager to temporarily add the path to sys.path
    with add_sys_path(path):
        # Import the module using its name
        module = importlib.import_module(module_name)
        # Reload the module to ensure any changes are reflected
        importlib.reload(module)
        # Return the imported and reloaded module
        return module


In [ ]:
# Setting working directory

# Define the path to the working directory
working_directory = '/Users/ruzejjur/Github/TMoCOBoT_python'

# Change the current working directory to the specified path
os.chdir(working_directory)

In [ ]:
# Add and import auxiliary script
auxiliary_path = os.path.abspath(os.path.join(working_directory, 'code', 'auxiliary'))
auxiliary = import_and_reload('auxiliary', auxiliary_path)

# Add and import testing script
testing_path = os.path.abspath(os.path.join(working_directory, 'code', 'auxiliary'))
testing = import_and_reload('testing', testing_path)

# Simulated examples

To simplify the analysis, specific parameters are defined for the experts, which include the following:

 - 10 experts for each brand. 
 - 3 mobile brands.
 - 3 features.
 - 6 score values.

These parameters have been deliberately chosen to configure the system and demonstrate the functionality of our proposed solution.

The parameters associated with the primary modeller are as follows:

 - The preference score $r_{I,n}$ = 4 for all features.
 - The primary modeller's brand $P_{I}(B)$ preference follows a uniform distribution.
 - The primary modeller's opinion on brands $b\in{B}$ is established in the experimental setup.
 - The primary modeller's certainty $c_{I,b,n} \in \langle 0, 1 \rangle$ is established in the experimental setup.
 - The primary modeller's trust $t_{I,E_{i}} \in \langle 0, 1 \rangle$ is established in the experimental setup.

The preference score is set for simplicity, eliminating one hyperparameter to tune. 
The primary modeller's brand preference $P_{I}(B)$ is configured to ensure that the choice of the brand $b\in{B}$ is not influenced by the primary modeller's bias.

The objective is to design experiments in such a way that the brand selection process can be inferred from the setup of the experts.

In the following five examples, the primary modeller's opinions are presented in the following table:

In [ ]:
# The primary modeller's opinion

# Array representing the scores given by the primary modeller for different phone brands and features
# * Note 1: This array represents primary modellers opinion
primary_modeller_scores = np.array([
    [5, 4, 5],  # Scores for Brand_1
    [6, 5, 6],  # Scores for Brand_2
    [4, 4, 3]   # Scores for Brand_3
], dtype=np.int8)


In [ ]:
# Creating a DataFrame with the given data
# Index represents phone brands, and columns represent different features
primary_modeller_scores_df = pd.DataFrame(
    primary_modeller_scores,
    index=["Brand 1", "Brand 2", "Brand 3"],  # Setting the row labels for phone brands
    columns=["Feature 1", "Feature 2", "Feature 3"]  # Setting the column labels for features
)

# Display the DataFrame
primary_modeller_scores_df

The primary modeller's opinion on brands is as follows: **Brand_2 > Brand_1 > Brand_3**.

## Opinion Merging and Preference Sub-setting

The aim of this section is to illustrate the influence of expert opinions on the primary modeller's brand preference.

### Example 1
The experts are configured to prefer the brands in the following order:

 **Brand_1 > Brand_2 > Brand_3**.

The Brand_1 is slightly more preferred than Brand_2, this is set up for later demonstration of the trust value $t_{I,E_{i}}$ in 'Inclusion of trust' section.

#### Experiment setup

##### Agent

In [ ]:
# Converting the primary modeller's scores to a DataFrame for better readability

# Creating a DataFrame with the given data
# Index represents phone brands, and columns represent different features
primary_modeller_scores_df = pd.DataFrame(
    primary_modeller_scores,
    index=["Brand 1", "Brand 2", "Brand 3"],  # Setting the row labels for phone brands
    columns=["Feature 1", "Feature 2", "Feature 3"]  # Setting the column labels for features
)

# Printing the DataFrame
print("The Primary Modeller's Scores:")
primary_modeller_scores_df

In [ ]:
# Setting up the opinion certainty

# Array representing the certainty of the primary modeller's opinion for each phone brand
# ! Note 1: ones does not equate to no certainty being applied as the opinion certainty is multiplied with number of responders
opinion_certainty_array = np.array([1, 1, 1], dtype=np.float16)

# Boolean flag to determine if certainty should be applied
# * Note 1: This flag is important as setting it to True will ensure that the opinion certainty array is ones.
apply_certainty = False


In [ ]:
# Setting up the number of responders for each brand so that the opinion
# certainty can be applied as many times as the responders responded

# Array representing the number of responders for each phone brand
number_of_responders = np.array([10, 10, 10], dtype=np.int32)

In [ ]:
# Create a trust matrix with ones, representing equal trust for each responder
# The matrix has 3 rows (one for each phone brand) and 10 columns (one for each responder)
# * Note 1: Setting trust like this is equivalent to no inclusion of trust
trust_matrix = np.ones((3, 10),dtype=np.float16)

In [ ]:
# Primary modeller's score preference for each feature

# Array representing the primary modeller's score preference for each feature
# * Note 1: This means that scores below the selected scores are not taken into acoount durring the opinion formulation
primary_modeller_score_preference = np.array([4, 4, 4], dtype=np.int8)

In [ ]:
# Primary modeller's brand preference

# Array representing the primary modeller's initial brand preference
# * Note 1: Set to uniform
primary_modeller_brand_pref = np.array([1, 1, 1], dtype=np.int8)

# Normalize the brand preference so that the sum equals 1
primary_modeller_brand_pref = primary_modeller_brand_pref / np.sum(primary_modeller_brand_pref)


In [ ]:
# Setting score range

# This represents the highest possible score for each feature, i.e., score range = 'best'
score_range = 6

In [ ]:
# The values of exponents in Dirichlet distribution must be non-zero,
# setting them to a low value

# Initial feature weight for the Dirichlet distribution
initial_feature_weight = 0.01


#### Expert opinions

The opinion's of individual experts $E_{i}$ for each brand $b\in{B}$.

In [ ]:
# Simulated experts for Brand_1
# Array representing the opinions of 10 simulated experts on Brand_1 for 3 features
Brand_1_expert_opinions = np.array([
    [4, 3, 4], [5, 3, 5], [5, 6, 5], [6, 5, 3], [6, 6, 6], 
    [5, 6, 5], [6, 6, 5], [6, 3, 4], [4, 5, 4], [6, 4, 3]
], dtype=np.int8)

# Simulated experts for Brand_2
# Array representing the opinions of 10 simulated experts on Brand_2 for 3 features
Brand_2_expert_opinions = np.array([
    [5, 5, 5], [5, 6, 5], [3, 4, 4], [3, 4, 5], [4, 5, 5], 
    [5, 6, 4], [6, 6, 6], [5, 6, 6], [4, 3, 4], [4, 6, 3]
], dtype=np.int8)

# Simulated experts for Brand_3
# Array representing the opinions of 10 simulated experts on Brand_3 for 3 features
Brand_3_expert_opinions = np.array([
    [3, 4, 3], [3, 3, 4], [4, 3, 4], [3, 3, 3], [5, 4, 3], 
    [3, 5, 5], [4, 5, 6], [4, 3, 2], [3, 4, 3], [4, 3, 4]
], dtype=np.int8)


In [ ]:
# Creating DataFrames for each brand

# DataFrame for Brand_1 expert opinions
Brand_1_expert_opinions_df = pd.DataFrame(
    Brand_1_expert_opinions, 
    index=[f"Brand_1 expert {i+1}" for i in range(10)],  # Setting index as expert identifiers
    columns=["Feature 1 score", "Feature 2 score", "Feature 3 score"]  # Setting the column labels for features
)

# DataFrame for Brand_2 expert opinions
Brand_2_expert_opinions_df = pd.DataFrame(
    Brand_2_expert_opinions,
    index=[f"Brand_2 expert {i+1}" for i in range(10)],  # Setting index as expert identifiers
    columns=["Feature 1 score", "Feature 2 score", "Feature 3 score"]  # Setting the column labels for features
)

# DataFrame for Brand_3 expert opinions
Brand_3_expert_opinions_df = pd.DataFrame(
    Brand_3_expert_opinions, 
    index=[f"Brand_3 expert {i+1}" for i in range(10)],  # Setting index as expert identifiers
    columns=["Feature 1 score", "Feature 2 score", "Feature 3 score"]  # Setting the column labels for features
)


In [ ]:
print("Brand_1 Experts' Opinions:")
Brand_1_expert_opinions_df

In [ ]:
print("\nBrand_2 Experts' Opinions:")
Brand_2_expert_opinions_df

In [ ]:
print("\nBrand_3 Experts' Opinions:")
Brand_3_expert_opinions_df

<ins>Simple inference of experts opinion of brands</ins>

In [ ]:
# Calculate the mean scores of each expert
Brand_1_means = Brand_1_expert_opinions_df.mean(axis=1)
Brand_2_means = Brand_2_expert_opinions_df.mean(axis=1)
Brand_3_means = Brand_3_expert_opinions_df.mean(axis=1)

# Create a box plot for each brand
fig, ax = plt.subplots(figsize=(10, 6))

ax.boxplot([Brand_1_means, Brand_2_means, Brand_3_means], labels=["Brand 1", "Brand 2", "Brand 3"], showmeans=True)
ax.set_title('Box Plot of Mean Expert Scores for Each Brand', fontsize=15.5)
ax.set_ylabel('Mean Score')
ax.set_xlabel('Brand')

# Adding scatter plot of the observations
x_positions = [1, 2, 3]
for i, means in enumerate([Brand_1_means, Brand_2_means, Brand_3_means]):
    ax.scatter([x_positions[i]] * len(means), means, alpha=0.6, color='blue', s=5)

# Save the figure in .eps format

# save_path = os.path.join(working_directory, 'Bar_charts', 'brand_mean_scores_boxplot.eps')
# plt.tight_layout()
# plt.savefig(save_path, format='eps')

# Show the plot
plt.show()


<ins>The primary modeller's brand preference after opinion merging.</ins>

In [ ]:
# Running the simulated example from the auxiliary module

# Call the simulated_example function with the provided parameters
_, primary_modeller_posterior_updated = auxiliary.simulated_example(
    primary_modeller_scores,  # Initial scores given by the primary modeller
    opinion_certainty_array,  # Certainty of the primary modeller's opinions
    apply_certainty,  # Flag to apply certainty
    number_of_responders,  # Number of responders for each brand
    trust_matrix,  # Trust matrix representing trust for each responder
    primary_modeller_score_preference,  # Primary modeller's score preference for each feature
    primary_modeller_brand_pref,  # Distribution representing primary modeller's brand prefference
    score_range,  # Highest possible score for each feature
    initial_feature_weight,  # Initial feature weight for the Dirichlet distribution
    Brand_1_expert_opinions,  # Expert opinions for Brand_1
    Brand_2_expert_opinions,  # Expert opinions for Brand_2
    Brand_3_expert_opinions  # Expert opinions for Brand_3
)


In [ ]:
## Plot primary modeller's updated posterior distribution on brands

# Variable names for the x-ticks
variable_names = ['Brand 1', 'Brand 2', 'Brand 3']

# Create a figure window with a single subplot
fig, ax = plt.subplots(1, 1, figsize=(7, 5))

# Plotting the updated posterior distribution as a bar chart
ax.bar(variable_names, primary_modeller_posterior_updated)
ax.set_ylabel('Probability')
ax.set_title("Primary modeller's updated brand preference", fontsize=15.5)
ax.tick_params(axis='x', labelsize=17)

# Save the figure in .eps format
save_path = os.path.join(working_directory, 'Bar_charts', 'figure_0.eps')
plt.tight_layout()
plt.savefig(save_path, format='eps')

# Show the plot
plt.show()


<ins>The primary modeller's updated brand preference.</ins>

In [ ]:
# Round the values and convert them to percentage strings
primary_modeller_posterior_updated_percent = [f"{value * 100:.2f}%" for value in primary_modeller_posterior_updated]

# Create a DataFrame for the updated posterior percentages
primary_modeller_posterior_updated_df = pd.DataFrame(
    [primary_modeller_posterior_updated_percent],    # Percentage values as a list
    columns=["Brand 1", "Brand 2", "Brand 3"]         # Column labels for the DataFrame
)

# Display the DataFrame
print("The Primary modeller's updated brand preference:")
primary_modeller_posterior_updated_df

The results suggest that the experts influenced the primary modeller's opinion to favour the Brand_1 brand. Additionally, the primary modeller's probability of choosing the Brand_3 brand prior to opinion merging is low due to the low score assigned to feature 3 for the Brand_3 brand.  Consequently, this leads to a low probability entering the posterior distribution before opinion merging, significantly reducing the probability of selecting the Brand_3 brand.

### Example 2

#### Experiment setup

##### Agent

In [ ]:
# Creating a DataFrame with the given data
# Index represents phone brands, and columns represent different features
primary_modeller_scores_df = pd.DataFrame(
    primary_modeller_scores,
    index=["Brand 1", "Brand 2", "Brand 3"],           # Setting the row labels for phone brands
    columns=["Feature 1", "Feature 2", "Feature 3"]  # Setting the column labels for features
)

# Display the DataFrame
primary_modeller_scores_df

In [ ]:
# Setting up the opinion certainty

# Array representing the certainty of the primary modeller's opinion for each phone brand
# ! Note 1: ones does not equate to no certainty being applied as the opinion certainty is multiplied with number of responders
opinion_certainty_array = np.array([1, 1, 1], dtype = np.float16)

# Boolean flag to determine if certainty should be applied
# * Note 1: This flag is important as setting it to True will ensure that the opinion certainty array is ones.
apply_certainty = False

In [ ]:
# Setting up the number of responders for each brand so that the opinion
# certainty can be applied as many times as the responders responded

# Array representing the number of responders for each phone brand
number_of_responders = np.array([10, 10, 10], dtype = np.int32)

In [ ]:
# Create a trust matrix with ones, representing equal trust for each responder
# The matrix has 3 rows (one for each phone brand) and 10 columns (one for each responder)
# * Note 1: Setting trust like this is equivalent to no inclusion of trust
trust_matrix = np.ones((3, 10), dtype=np.float16)

In [ ]:
# Primary modeller's score preference for each feature

# Array representing the primary modeller's score preference for each feature
# * Note 1: This means that scores below the selected scores are not taken into acoount durring the opinion formulation
primary_modeller_score_preference = np.array([4, 4, 4], dtype=np.int8)

In [ ]:
# Primary modeller's brand preference

# Array representing the primary modeller's initial brand preference
# * Note 1: Set to uniform
primary_modeller_brand_pref = np.array([1, 1, 1], dtype=np.float16)

# Normalize the brand preference so that the sum equals 1
primary_modeller_brand_pref = primary_modeller_brand_pref / np.sum(primary_modeller_brand_pref)

In [ ]:
# Setting score range

# This represents the highest possible score for each feature, i.e., score range = 'best'
score_range = 6

In [ ]:
# The values of exponents in Dirichlet distribution must be non-zero,
# setting them to a low value

# Initial feature weight for the Dirichlet distribution
initial_feature_weight = 0.01

##### Expert opinons

The experts are configured to prefer the brands in the following order:

 **Brand_1 $\approx$ Brand_2 $\approx$ Brand_3**, 

the scores are set up to similar **<ins>high</ins>** values.

In [ ]:
# Simulated experts for Brand_1
# Array representing the opinions of 10 simulated experts on Brand_1 for 3 features
Brand_1_expert_opinions = np.array([
    [6, 5, 6], [5, 5, 5], [6, 5, 5], [5, 5, 6], [6, 6, 6],
    [5, 6, 5], [6, 5, 5], [6, 5, 5], [5, 6, 6], [6, 5, 5]
], dtype=np.int8)

# Simulated experts for Brand_2
# Array representing the opinions of 10 simulated experts on Brand_2 for 3 features
Brand_2_expert_opinions = np.array([
    [5, 6, 5], [6, 5, 5], [6, 5, 6], [5, 6, 5], [6, 6, 5],
    [6, 5, 6], [6, 6, 6], [6, 5, 6], [5, 6, 5], [6, 6, 6]
], dtype=np.int8)

# Simulated experts for Brand_3
# Array representing the opinions of 10 simulated experts on Brand_3 for 3 features
Brand_3_expert_opinions = np.array([
    [5, 5, 6], [6, 5, 6], [6, 6, 6], [6, 6, 6], [6, 5, 6],
    [6, 5, 5], [6, 5, 6], [5, 5, 6], [6, 6, 5], [5, 6, 5]
], dtype=np.int8)

In [ ]:
# Creating DataFrames for each brand

# DataFrame for Brand_1 expert opinions
Brand_1_expert_opinions_df = pd.DataFrame(
    Brand_1_expert_opinions, 
    index=[f"Brand_1 expert {i+1}" for i in range(10)],  # Setting index as expert identifiers
    columns=["Feature 1 score", "Feature 2 score", "Feature 3 score"]  # Setting the column labels for features
)

# DataFrame for Brand_2 expert opinions
Brand_2_expert_opinions_df = pd.DataFrame(
    Brand_2_expert_opinions,
    index=[f"Brand_2 expert {i+1}" for i in range(10)],  # Setting index as expert identifiers
    columns=["Feature 1 score", "Feature 2 score", "Feature 3 score"]  # Setting the column labels for features
)

# DataFrame for Brand_3 expert opinions
Brand_3_expert_opinions_df = pd.DataFrame(
    Brand_3_expert_opinions, 
    index=[f"Brand_3 expert {i+1}" for i in range(10)],  # Setting index as expert identifiers
    columns=["Feature 1 score", "Feature 2 score", "Feature 3 score"]  # Setting the column labels for features
)


<ins>Simple inference of experts opinion of brands</ins>

In [ ]:
# Calculate the mean scores of each expert
Brand_1_means = Brand_1_expert_opinions_df.mean(axis=1)
Brand_2_means = Brand_2_expert_opinions_df.mean(axis=1)
Brand_3_means = Brand_3_expert_opinions_df.mean(axis=1)

# Create a box plot for each brand
fig, ax = plt.subplots(figsize=(10, 6))

ax.boxplot([Brand_1_means, Brand_2_means, Brand_3_means], labels=["Brand 1", "Brand 2", "Brand 3"], showmeans=True)
ax.set_title('Box Plot of Mean Expert Scores for Each Brand', fontsize=15.5)
ax.set_ylabel('Mean Score')
ax.set_xlabel('Brand')

# Adding scatter plot of the observations
x_positions = [1, 2, 3]
for i, means in enumerate([Brand_1_means, Brand_2_means, Brand_3_means]):
    ax.scatter([x_positions[i]] * len(means), means, alpha=0.6, color='blue', s=5)

# Save the figure in .eps format
# save_path = os.path.join(working_directory, 'Bar_charts', 'brand_mean_scores_boxplot.eps')
# plt.tight_layout()
# plt.savefig(save_path, format='eps')

# Show the plot
plt.show()


In [ ]:
# Running the simulated example from the auxiliary module

# Call the simulated_example function with the provided parameters
_, primary_modeller_posterior_updated = auxiliary.simulated_example(
    primary_modeller_scores,  # Initial scores given by the primary modeller
    opinion_certainty_array,  # Certainty of the primary modeller's opinions
    apply_certainty,  # Flag to apply certainty
    number_of_responders,  # Number of responders for each brand
    trust_matrix,  # Trust matrix representing trust for each responder
    primary_modeller_score_preference,  # Primary modeller's score preference for each feature
    primary_modeller_brand_pref,  # Distribution representing primary modeller's brand prefference
    score_range,  # Highest possible score for each feature
    initial_feature_weight,  # Initial feature weight for the Dirichlet distribution
    Brand_1_expert_opinions,  # Expert opinions for Brand_1
    Brand_2_expert_opinions,  # Expert opinions for Brand_2
    Brand_3_expert_opinions  # Expert opinions for Brand_3
)


In [ ]:
# Create a figure window
plt.figure(figsize=(7, 5))

# Plot the primary modeller's updated posterior distribution on brands
plt.bar(['Brand 1', 'Brand 2', 'Brand 3'], primary_modeller_posterior_updated)

# Set the font size for the x-ticks
plt.xticks(fontsize=17)

# Label the y-axis and set the title
plt.ylabel('Probability')
plt.title("Primary modeller's updated brand preference", fontsize=18)

# Save the plot
save_path = os.path.join(working_directory, 'Bar_charts', 'figure_1.eps')
plt.tight_layout()
plt.savefig(save_path, format='eps')

# Display the plot
plt.show()


<ins>The primary modeller's updated brand preference.</ins>

In [ ]:
# Round the values and convert them to percentage strings
primary_modeller_posterior_updated_percent = [f"{value * 100:.2f}%" for value in primary_modeller_posterior_updated]

# Create a DataFrame for the updated posterior percentages
primary_modeller_posterior_updated_df = pd.DataFrame(
    [primary_modeller_posterior_updated_percent],    # Percentage values as a list
    columns=["Brand 1", "Brand 2", "Brand 3"]         # Column labels for the DataFrame
)

# Display the DataFrame
print("The Primary modeller's updated brand preference:")
primary_modeller_posterior_updated_df

Comment on this result are at the end of this section.

### Example 3

The experts are configured to prefer the brands in the following order:

**Brand_1 $\approx$ Brand_2 $\approx$ Brand_3**, 
                      
the scores are set up to similar **<ins>low</ins>** values.

#### Experiment setup

##### Agent

In [ ]:
# Creating a DataFrame with the given data
# Index represents phone brands, and columns represent different features
primary_modeller_scores_df = pd.DataFrame(
    primary_modeller_scores,
    index=["Brand 1", "Brand 2", "Brand 3"],           # Setting the row labels for phone brands
    columns=["Feature 1", "Feature 2", "Feature 3"]  # Setting the column labels for features
)

# Display the DataFrame
primary_modeller_scores_df

In [ ]:
# Setting up the opinion certainty

# Array representing the certainty of the primary modeller's opinion for each phone brand
# ! Note 1: ones does not equate to no certainty being applied as the opinion certainty is multiplied with number of responders
opinion_certainty_array = np.array([1, 1, 1], dtype=np.float16)

# Boolean flag to determine if certainty should be applied
# * Note 1: This flag is important as setting it to True will ensure that the opinion certainty array is ones.
apply_certainty = False


In [ ]:
# Setting up the number of responders for each brand so that the opinion
# certainty can be applied as many times as the responders responded

# Array representing the number of responders for each phone brand
number_of_responders = np.array([10, 10, 10], dtype=np.int32)

In [ ]:
# Create a trust matrix with ones, representing equal trust for each responder
# The matrix has 3 rows (one for each phone brand) and 10 columns (one for each responder)
# * Note 1: Setting trust like this is equivalent to no inclusion of trust
trust_matrix = np.ones((3, 10), dtype=np.float16)

In [ ]:
# Primary modeller's score preference for each feature

# Array representing the primary modeller's score preference for each feature
# * Note 1: This means that scores below the selected scores are not taken into acoount durring the opinion formulation
primary_modeller_score_preference = np.array([4, 4, 4], dtype=np.int8)

In [ ]:
# Primary modeller's brand preference

# Array representing the primary modeller's initial brand preference
# * Note 1: Set to uniform
primary_modeller_brand_pref = np.array([1, 1, 1], dtype=np.float32)

# Normalize the brand preference so that the sum equals 1
primary_modeller_brand_pref = primary_modeller_brand_pref / np.sum(primary_modeller_brand_pref)


In [ ]:
# Setting score range

# This represents the highest possible score for each feature, i.e., score range = 'best'
score_range = 6

In [ ]:
# The values of exponents in Dirichlet distribution must be non-zero,
# setting them to a low value

# Initial feature weight for the Dirichlet distribution
initial_feature_weight = 0.01


#### Expert opinions

In [ ]:
# Simulated experts for Brand_1
# Array representing the opinions of 10 simulated experts on Brand_1 for 3 features
Brand_1_expert_opinions = np.array([
    [3, 4, 4], [3, 4, 4], [4, 4, 4], [4, 3, 3], [4, 3, 3],
    [4, 4, 1], [4, 1, 4], [4, 3, 4], [4, 5, 4], [4, 4, 3]
], dtype=np.int8)

# Simulated experts for Brand_2
# Array representing the opinions of 10 simulated experts on Brand_2 for 3 features
Brand_2_expert_opinions = np.array([
    [4, 4, 4], [4, 3, 4], [4, 4, 4], [4, 4, 2], [4, 3, 4],
    [3, 4, 1], [4, 4, 4], [2, 4, 4], [1, 4, 4], [3, 2, 1]
], dtype=np.int8)

# Simulated experts for Brand_3
# Array representing the opinions of 10 simulated experts on Brand_3 for 3 features
Brand_3_expert_opinions = np.array([
    [4, 4, 4], [4, 4, 4], [3, 5, 4], [4, 4, 4], [2, 1, 4],
    [4, 4, 5], [3, 4, 2], [4, 3, 4], [4, 4, 1], [1, 2, 3]
], dtype=np.int8)


In [ ]:
# Creating DataFrames for each brand

# DataFrame for Brand_1 expert opinions
Brand_1_expert_opinions_df = pd.DataFrame(
    Brand_1_expert_opinions, 
    index=[f"Brand_1 expert {i+1}" for i in range(10)],  # Setting index as expert identifiers
    columns=["Feature 1 score", "Feature 2 score", "Feature 3 score"]  # Setting the column labels for features
)

# DataFrame for Brand_2 expert opinions
Brand_2_expert_opinions_df = pd.DataFrame(
    Brand_2_expert_opinions,
    index=[f"Brand_2 expert {i+1}" for i in range(10)],  # Setting index as expert identifiers
    columns=["Feature 1 score", "Feature 2 score", "Feature 3 score"]  # Setting the column labels for features
)

# DataFrame for Brand_3 expert opinions
Brand_3_expert_opinions_df = pd.DataFrame(
    Brand_3_expert_opinions, 
    index=[f"Brand_3 expert {i+1}" for i in range(10)],  # Setting index as expert identifiers
    columns=["Feature 1 score", "Feature 2 score", "Feature 3 score"]  # Setting the column labels for features
)


<ins>Simple inference of experts opinion of brands</ins>

In [ ]:
# Calculate the mean scores of each expert
Brand_1_means = Brand_1_expert_opinions_df.mean(axis=1)
Brand_2_means = Brand_2_expert_opinions_df.mean(axis=1)
Brand_3_means = Brand_3_expert_opinions_df.mean(axis=1)

# Create a box plot for each brand
fig, ax = plt.subplots(figsize=(10, 6))

ax.boxplot([Brand_1_means, Brand_2_means, Brand_3_means], labels=["Brand 1", "Brand 2", "Brand 3"], showmeans=True)
ax.set_title('Box Plot of Mean Expert Scores for Each Brand', fontsize=15.5)
ax.set_ylabel('Mean Score')
ax.set_xlabel('Brand')

# Adding scatter plot of the observations
x_positions = [1, 2, 3]
for i, means in enumerate([Brand_1_means, Brand_2_means, Brand_3_means]):
    ax.scatter([x_positions[i]] * len(means), means, alpha=0.6, color='blue', s=5)

# Save the figure in .eps format

# save_path = os.path.join(working_directory, 'Bar_charts', 'brand_mean_scores_boxplot.eps')
# plt.tight_layout()
# plt.savefig(save_path, format='eps')

# Show the plot
plt.show()


In [ ]:
# Running the simulated example from the auxiliary module

# Call the simulated_example function with the provided parameters
_, primary_modeller_posterior_updated = auxiliary.simulated_example(
    primary_modeller_scores,  # Initial scores given by the primary modeller
    opinion_certainty_array,  # Certainty of the primary modeller's opinions
    apply_certainty,  # Flag to apply certainty
    number_of_responders,  # Number of responders for each brand
    trust_matrix,  # Trust matrix representing trust for each responder
    primary_modeller_score_preference,  # Primary modeller's score preference for each feature
    primary_modeller_brand_pref,  # Distribution representing primary modeller's brand prefference
    score_range,  # Highest possible score for each feature
    initial_feature_weight,  # Initial feature weight for the Dirichlet distribution
    Brand_1_expert_opinions,  # Expert opinions for Brand_1
    Brand_2_expert_opinions,  # Expert opinions for Brand_2
    Brand_3_expert_opinions  # Expert opinions for Brand_3
)


In [ ]:
# Create a figure window
plt.figure(figsize=(7, 5))

# Plot the primary modeller's updated posterior distribution on brands
plt.bar(['Brand 1', 'Brand 2', 'Brand 3'], primary_modeller_posterior_updated)

# Set the font size for the x-ticks
plt.xticks(fontsize=17)

# Label the y-axis and set the title
plt.ylabel('Probability')
plt.title("Primary modeller's updated brand preference", fontsize=18)

# Save the plot
save_path = os.path.join(working_directory, 'Bar_charts', 'figure_2.eps')
plt.tight_layout()
plt.savefig(save_path, format='eps')

# Display the plot
plt.show()

<ins>The primary modeller's updated brand preference.</ins>

In [ ]:
# Round the values and convert them to percentage strings
primary_modeller_posterior_updated_percent = [f"{value * 100:.2f}%" for value in primary_modeller_posterior_updated]

# Create a DataFrame for the updated posterior percentages
primary_modeller_posterior_updated_df = pd.DataFrame(
    [primary_modeller_posterior_updated_percent],    # Percentage values as a list
    columns=["Brand 1", "Brand 2", "Brand 3"]         # Column labels for the DataFrame
)

# Display the DataFrame
print("The Primary modeller's updated brand preference:")
primary_modeller_posterior_updated_df

The results of the last two experiments show only a minor difference in the final brand preference. However, it was expected that the primary modeller's opinion would have a stronger influence, pushing the preference towards the following order: **Brand_2 > Brand_1 > Brand_3**.

This is attributed to the fact that each expert's contribution to the updated weight $V_{f_{j,b}}$ has a magnitude of +1, and the primary modeller's contribution is also + 1. The cumulative effect of the experts' contributions diminishes the impact of the primary modeller's opinion.

To address this, the magnitude of the primary modeller's weights $n_{f_{1,b}}$ needs to be adjusted to ensure that the primary modeller's opinion is not diminished. Further elaboration on this adjustment will be provided in the section titled 'Inclusion of Opinion Certainty' below.

## Inclusion of trust
In this section, we are examining the integration of trust $t_{I,E_{i}}$, into each expert's $E_{i}$ opinion.  The previously mentioned issue still exists and will be addressed later.
For now, the primary modeller's opinion will be de-emphasized. This approach allows for a more precise demonstration of trust inclusion, free from any bias introduced by the primary modeller's opinion.
For simplicity, the setup of the following two experiments is the same as in the first example from the previous section.

### Example 1
We start by configuring low trust values $t_{I,E_{i}}$ for experts reacting to the Brand_1 brand and high trust values for experts reacting to Brand_2 and Brand_3. The expected outcome is that the preferred brand should be Brand_2, since it has similar score values provided by the experts' $E_{i}$ as Brand_1, with Brand_2 being slightly less favoured.

#### Experiment setup

##### Agent

In [ ]:
# The primary modeller's opinion

# Array representing the scores given by the primary modeller for different phone brands and features
# * Note 1: This array represents primary modellers opinion
primary_modeller_scores = np.array([
    [5, 4, 5],  # Scores for Brand_1
    [6, 5, 6],  # Scores for Brand_2
    [4, 4, 3]   # Scores for Brand_3
], dtype=np.int8)

In [ ]:
# Creating a DataFrame with the given data
# Index represents phone brands, and columns represent different features
primary_modeller_scores_df = pd.DataFrame(
    primary_modeller_scores,
    index=["Brand 1", "Brand 2", "Brand 3"],           # Setting the row labels for phone brands
    columns=["Feature 1", "Feature 2", "Feature 3"]  # Setting the column labels for features
)

# Display the DataFrame
primary_modeller_scores_df

In [ ]:
# Setting up the opinion certainty

# Array representing the certainty of the primary modeller's opinion for each phone brand
# ! Note 1: ones does not equate to no certainty being applied as the opinion certainty is multiplied with number of responders
opinion_certainty_array = np.array([1, 1, 1], dtype=np.float16)

# Boolean flag to determine if certainty should be applied
# * Note 1: This flag is important as setting it to True will ensure that the opinion certainty array is ones.
apply_certainty = False

In [ ]:
# Setting up the number of responders for each brand so that the opinion
# certainty can be applied as many times as the responders responded

# Array representing the number of responders for each phone brand
number_of_responders = np.array([10, 10, 10], dtype=np.int32)

In [ ]:
# Setting up trust in experts' opinions

# Trust matrix representing the trust levels for each expert's opinion
# Rows represent phone brands (Brand_1, Brand_2, Brand_3)
# Columns represent individual experts' trust levels
trust_matrix = np.array([
    [0.8, 0.9, 0.3, 0.7, 0.2, 0.2, 0.3, 0.5, 0.4, 0.9],  # Trust levels for Brand_1 experts
    [0.9, 0.9, 0.3, 0.3, 0.7, 0.7, 0.9, 0.9, 0.4, 0.4],  # Trust levels for Brand_2 experts
    [0.9, 0.7, 0.8, 0.8, 0.9, 0.8, 0.7, 0.7, 0.8, 0.9]   # Trust levels for Brand_3 experts
], dtype=np.float32)

In [ ]:
# Primary modeller's score preference for each feature

# Array representing the primary modeller's score preference for each feature
# * Note 1: This means that scores below the selected scores are not taken into acoount durring the opinion formulation
primary_modeller_score_preference = np.array([4, 4, 4], dtype=np.int8)

In [ ]:
# Primary modeller's brand preference

# Array representing the primary modeller's initial brand preference
# * Note 1: Set to uniform
primary_modeller_brand_pref = np.array([1, 1, 1], dtype=np.float32)

# Normalize the brand preference so that the sum equals 1
primary_modeller_brand_pref = primary_modeller_brand_pref / np.sum(primary_modeller_brand_pref)

In [ ]:
# Setting score range

# This represents the highest possible score for each feature, i.e., score range = 'best'
score_range = 6

In [ ]:
# The values of exponents in Dirichlet distribution must be non-zero,
# setting them to a low value

# Initial feature weight for the Dirichlet distribution
initial_feature_weight = 0.01

#### Expert opinions

The opinion's of individual experts $E_{i}$ for each brand $b\in{B}$.

In [ ]:
# Simulated experts for Brand_1
# Array representing the opinions of 10 simulated experts on Brand_1 for 3 features
Brand_1_expert_opinions = np.array([
    [4, 3, 4], [5, 3, 5], [5, 6, 5], [6, 5, 3], [6, 6, 6], 
    [5, 6, 5], [6, 6, 5], [6, 3, 4], [4, 5, 4], [6, 4, 3]
], dtype=np.int8)

# Simulated experts for Brand_2
# Array representing the opinions of 10 simulated experts on Brand_2 for 3 features
Brand_2_expert_opinions = np.array([
    [5, 5, 5], [5, 6, 5], [3, 4, 4], [3, 4, 5], [4, 5, 5], 
    [5, 6, 4], [6, 6, 6], [5, 6, 6], [4, 3, 4], [4, 6, 3]
], dtype=np.int8)

# Simulated experts for Brand_3
# Array representing the opinions of 10 simulated experts on Brand_3 for 3 features
Brand_3_expert_opinions = np.array([
    [3, 4, 3], [3, 3, 4], [4, 3, 4], [3, 3, 3], [5, 4, 3], 
    [3, 5, 5], [4, 5, 6], [4, 3, 2], [3, 4, 3], [4, 3, 4]
], dtype=np.int8)

In [ ]:
# Creating DataFrames for each brand

# DataFrame for Brand_1 expert opinions
Brand_1_expert_opinions_df = pd.DataFrame(
    Brand_1_expert_opinions, 
    index=[f"Brand_1 expert {i+1}" for i in range(10)],  # Setting index as expert identifiers
    columns=["Feature 1 score", "Feature 2 score", "Feature 3 score"]  # Setting the column labels for features
)

# DataFrame for Brand_2 expert opinions
Brand_2_expert_opinions_df = pd.DataFrame(
    Brand_2_expert_opinions,
    index=[f"Brand_2 expert {i+1}" for i in range(10)],  # Setting index as expert identifiers
    columns=["Feature 1 score", "Feature 2 score", "Feature 3 score"]  # Setting the column labels for features
)

# DataFrame for Brand_3 expert opinions
Brand_3_expert_opinions_df = pd.DataFrame(
    Brand_3_expert_opinions, 
    index=[f"Brand_3 expert {i+1}" for i in range(10)],  # Setting index as expert identifiers
    columns=["Feature 1 score", "Feature 2 score", "Feature 3 score"]  # Setting the column labels for features
)


<ins>Simple inference of experts opinion of brands</ins>

In [ ]:
# Calculate the mean scores of each expert
Brand_1_means = Brand_1_expert_opinions_df.mean(axis=1)
Brand_2_means = Brand_2_expert_opinions_df.mean(axis=1)
Brand_3_means = Brand_3_expert_opinions_df.mean(axis=1)

# Create a box plot for each brand
fig, ax = plt.subplots(figsize=(10, 6))

ax.boxplot([Brand_1_means, Brand_2_means, Brand_3_means], labels=["Brand 1", "Brand 2", "Brand 3"], showmeans=True)
ax.set_title('Box Plot of Mean Expert Scores for Each Brand', fontsize=15.5)
ax.set_ylabel('Mean Score')
ax.set_xlabel('Brand')

# Adding scatter plot of the observations
x_positions = [1, 2, 3]
for i, means in enumerate([Brand_1_means, Brand_2_means, Brand_3_means]):
    ax.scatter([x_positions[i]] * len(means), means, alpha=0.6, color='blue', s=5)

# Save the figure in .eps format

# save_path = os.path.join(working_directory, 'Bar_charts', 'brand_mean_scores_boxplot.eps')
# plt.tight_layout()
# plt.savefig(save_path, format='eps')

# Show the plot
plt.show()


In [ ]:
# Running the simulated example from the auxiliary module

# Call the simulated_example function with the provided parameters
_, primary_modeller_posterior_updated = auxiliary.simulated_example(
    primary_modeller_scores,  # Initial scores given by the primary modeller
    opinion_certainty_array,  # Certainty of the primary modeller's opinions
    apply_certainty,  # Flag to apply certainty
    number_of_responders,  # Number of responders for each brand
    trust_matrix,  # Trust matrix representing trust for each responder
    primary_modeller_score_preference,  # Primary modeller's score preference for each feature
    primary_modeller_brand_pref,  # Distribution representing primary modeller's brand prefference
    score_range,  # Highest possible score for each feature
    initial_feature_weight,  # Initial feature weight for the Dirichlet distribution
    Brand_1_expert_opinions,  # Expert opinions for Brand_1
    Brand_2_expert_opinions,  # Expert opinions for Brand_2
    Brand_3_expert_opinions  # Expert opinions for Brand_3
)

In [ ]:
# Create a figure window
plt.figure(figsize=(7, 5))

# Plot the primary modeller's updated posterior distribution on brands
plt.bar(['Brand 1', 'Brand 2', 'Brand 3'], primary_modeller_posterior_updated)

# Set the font size for the x-ticks
plt.xticks(fontsize=17)

# Label the y-axis and set the title
plt.ylabel('Probability')
plt.title("Primary modeller's updated brand preference", fontsize=18)

# Save the plot
save_path = os.path.join(working_directory, 'Bar_charts', 'figure_3.eps')
plt.tight_layout()
plt.savefig(save_path, format='eps')

# Display the plot
plt.show()

<ins>The primary modeller's updated brand preference.</ins>

In [ ]:
# Round the values and convert them to percentage strings
primary_modeller_posterior_updated_percent = [f"{value * 100:.2f}%" for value in primary_modeller_posterior_updated]

# Create a DataFrame for the updated posterior percentages
primary_modeller_posterior_updated_df = pd.DataFrame(
    [primary_modeller_posterior_updated_percent],    # Percentage values as a list
    columns=["Brand 1", "Brand 2", "Brand 3"]         # Column labels for the DataFrame
)

# Display the DataFrame
print("The Primary modeller's updated brand preference:")
primary_modeller_posterior_updated_df

### Example 2
Setting the trust $t_{I,E_{i}}$ for experts reacting to Brand_1 and Brand_2 brands to low values and to high values for Brand_3. The preferred brand should be Brand_3.

#### Experiment setup

##### Agent

In [ ]:
# The primary modeller's opinion

# Array representing the scores given by the primary modeller for different phone brands and features
# * Note 1: This array represents primary modellers opinion
primary_modeller_scores = np.array([
    [5, 4, 5],  # Scores for Brand_1
    [6, 5, 6],  # Scores for Brand_2
    [4, 4, 3]   # Scores for Brand_3
], dtype=np.int8)

In [ ]:
# Creating a DataFrame with the given data
# Index represents phone brands, and columns represent different features
primary_modeller_scores_df = pd.DataFrame(
    primary_modeller_scores,
    index=["Brand 1", "Brand 2", "Brand 3"],  # Setting the row labels for phone brands
    columns=["Feature 1", "Feature 2", "Feature 3"]  # Setting the column labels for features
)

# Display the DataFrame
primary_modeller_scores_df

In [ ]:
# Setting up the opinion certainty

# Array representing the certainty of the primary modeller's opinion for each phone brand
# ! Note 1: ones does not equate to no certainty being applied as the opinion certainty is multiplied with number of responders
opinion_certainty_array = np.array([1, 1, 1], dtype=np.float16)

# Boolean flag to determine if certainty should be applied
# * Note 1: This flag is important as setting it to True will ensure that the opinion certainty array is ones.
apply_certainty = False

In [ ]:
# Setting up the number of responders for each brand so that the opinion
# certainty can be applied as many times as the responders responded

# Array representing the number of responders for each phone brand
number_of_responders = np.array([10, 10, 10], dtype=np.int32)

In [ ]:
# Setting up trust in experts' opinions

# Trust matrix representing the trust levels for each expert's opinion
# Rows represent phone brands (Brand_1, Brand_2, Brand_3)
# Columns represent individual experts' trust levels
trust_matrix = np.array([
    [0.8, 0.9, 0.5, 0.9, 0.2, 0.2, 0.2, 0.5, 0.4, 0.9],  # Trust levels for Brand_1 experts
    [0.3, 0.3, 0.9, 0.9, 0.5, 0.5, 0.2, 0.2, 0.8, 0.8],  # Trust levels for Brand_2 experts
    [0.5, 0.7, 0.9, 0.2, 0.9, 0.8, 0.9, 0.7, 0.8, 0.9]   # Trust levels for Brand_3 experts
], dtype=np.float32)

In [ ]:
# Primary modeller's score preference for each feature

# Array representing the primary modeller's score preference for each feature
# * Note 1: This means that scores below the selected scores are not taken into acoount durring the opinion formulation
primary_modeller_score_preference = np.array([4, 4, 4], dtype=np.int8)

In [ ]:
# Primary modeller's brand preference

# Array representing the primary modeller's initial brand preference
# * Note 1: Set to uniform
primary_modeller_brand_pref = np.array([1, 1, 1], dtype=np.float32)

# Normalize the brand preference so that the sum equals 1
primary_modeller_brand_pref = primary_modeller_brand_pref / np.sum(primary_modeller_brand_pref)

In [ ]:
# Setting score range

# This represents the highest possible score for each feature, i.e., score range = 'best'
score_range = 6

In [ ]:
# The values of exponents in Dirichlet distribution must be non-zero,
# setting them to a low value

# Initial feature weight for the Dirichlet distribution
initial_feature_weight = 0.01

##### Expert opinions

In [ ]:
# Simulated experts for Brand_1
# Array representing the opinions of 10 simulated experts on Brand_1 for 3 features
Brand_1_expert_opinions = np.array([
    [4, 3, 4], [5, 3, 5], [5, 6, 5], [6, 5, 3], [6, 6, 6], 
    [5, 6, 5], [6, 6, 5], [6, 3, 4], [4, 5, 4], [6, 4, 3]
], dtype=np.int8)

# Simulated experts for Brand_2
# Array representing the opinions of 10 simulated experts on Brand_2 for 3 features
Brand_2_expert_opinions = np.array([
    [5, 5, 5], [5, 6, 5], [3, 4, 4], [3, 4, 5], [4, 5, 5], 
    [5, 6, 4], [6, 6, 6], [5, 6, 6], [4, 3, 4], [4, 6, 3]
], dtype=np.int8)

# Simulated experts for Brand_3
# Array representing the opinions of 10 simulated experts on Brand_3 for 3 features
Brand_3_expert_opinions = np.array([
    [3, 4, 3], [3, 3, 4], [4, 3, 4], [3, 3, 3], [5, 4, 3], 
    [3, 5, 5], [4, 5, 6], [4, 3, 2], [3, 4, 3], [4, 3, 4]
], dtype=np.int8)


In [ ]:
# Creating DataFrames for each brand

# DataFrame for Brand_1 expert opinions
Brand_1_expert_opinions_df = pd.DataFrame(
    Brand_1_expert_opinions, 
    index=[f"Brand_1 expert {i+1}" for i in range(10)],  # Setting index as expert identifiers
    columns=["Feature 1 score", "Feature 2 score", "Feature 3 score"]  # Setting the column labels for features
)

# DataFrame for Brand_2 expert opinions
Brand_2_expert_opinions_df = pd.DataFrame(
    Brand_2_expert_opinions,
    index=[f"Brand_2 expert {i+1}" for i in range(10)],  # Setting index as expert identifiers
    columns=["Feature 1 score", "Feature 2 score", "Feature 3 score"]  # Setting the column labels for features
)

# DataFrame for Brand_3 expert opinions
Brand_3_expert_opinions_df = pd.DataFrame(
    Brand_3_expert_opinions, 
    index=[f"Brand_3 expert {i+1}" for i in range(10)],  # Setting index as expert identifiers
    columns=["Feature 1 score", "Feature 2 score", "Feature 3 score"]  # Setting the column labels for features
)


<ins>Simple inference of experts opinion of brands</ins>

In [ ]:
# Calculate the mean scores of each expert
Brand_1_means = Brand_1_expert_opinions_df.mean(axis=1)
Brand_2_means = Brand_2_expert_opinions_df.mean(axis=1)
Brand_3_means = Brand_3_expert_opinions_df.mean(axis=1)

# Create a box plot for each brand
fig, ax = plt.subplots(figsize=(10, 6))

ax.boxplot([Brand_1_means, Brand_2_means, Brand_3_means], labels=["Brand 1", "Brand 2", "Brand 3"], showmeans=True)
ax.set_title('Box Plot of Mean Expert Scores for Each Brand', fontsize=15.5)
ax.set_ylabel('Mean Score')
ax.set_xlabel('Brand')

# Adding scatter plot of the observations
x_positions = [1, 2, 3]
for i, means in enumerate([Brand_1_means, Brand_2_means, Brand_3_means]):
    ax.scatter([x_positions[i]] * len(means), means, alpha=0.6, color='blue', s=5)

# Save the figure in .eps format

# save_path = os.path.join(working_directory, 'Bar_charts', 'brand_mean_scores_boxplot.eps')
# plt.tight_layout()
# plt.savefig(save_path, format='eps')

# Show the plot
plt.show()


In [ ]:
# Running the simulated example from the auxiliary module

# Call the simulated_example function with the provided parameters
_, primary_modeller_posterior_updated = auxiliary.simulated_example(
    primary_modeller_scores,  # Initial scores given by the primary modeller
    opinion_certainty_array,  # Certainty of the primary modeller's opinions
    apply_certainty,  # Flag to apply certainty
    number_of_responders,  # Number of responders for each brand
    trust_matrix,  # Trust matrix representing trust for each responder
    primary_modeller_score_preference,  # Primary modeller's score preference for each feature
    primary_modeller_brand_pref,  # Distribution representing primary modeller's brand prefference
    score_range,  # Highest possible score for each feature
    initial_feature_weight,  # Initial feature weight for the Dirichlet distribution
    Brand_1_expert_opinions,  # Expert opinions for Brand_1
    Brand_2_expert_opinions,  # Expert opinions for Brand_2
    Brand_3_expert_opinions  # Expert opinions for Brand_3
)


In [ ]:
# Create a figure window
plt.figure(figsize=(7, 5))

# Plot the primary modeller's updated posterior distribution on brands
plt.bar(['Brand 1', 'Brand 2', 'Brand 3'], primary_modeller_posterior_updated)

# Set the font size for the x-ticks
plt.xticks(fontsize=17)

# Label the y-axis and set the title
plt.ylabel('Probability')
plt.title("Primary modeller's updated brand preference", fontsize=18)

# Save the plot
save_path = os.path.join(working_directory, 'Bar_charts', 'figure_4.eps')
plt.tight_layout()
plt.savefig(save_path, format='eps')

# Display the plot
plt.show()

<ins>The primary modeller's updated brand preference.</ins>

In [ ]:
# Round the values and convert them to percentage strings
primary_modeller_posterior_updated_percent = [f"{value * 100:.2f}%" for value in primary_modeller_posterior_updated]

# Create a DataFrame for the updated posterior percentages
primary_modeller_posterior_updated_df = pd.DataFrame(
    [primary_modeller_posterior_updated_percent],    # Percentage values as a list
    columns=["Brand 1", "Brand 2", "Brand 3"]         # Column labels for the DataFrame
)

# Display the DataFrame
print("The Primary modeller's updated brand preference:")
primary_modeller_posterior_updated_df

The trust process works as intended.

### Inclusion of certainty
This section aims to provide a solution to the problem of setting the primary modeller's initial weights $n_{f_{1,b}}$ so that the primary modeller's opinion is not diminished, this is described in the first three examples. 
To better demonstrate the proposed solution, the trust values $t_{I,E_{i}} \in \langle 0, 1 \rangle$ are intentionally excluded. When the trust value is not set, it is equivalent to setting the trust $t_{I,E_{i}}$ to the maximum value of 1.
In contrast, in the opposite scenario, the weight increments of the experts are generally $t_{I,E_{i}} \in \langle 0, 1 \rangle$, this effect is linearly combined with the value of opinion certainty $c_{I,b,n}$.
A simplified solution was deemed to be sufficient.

For simplicity, the setup of the following three examples are the same as in the first example from the first section, apart from the setup of the primary modeller's opinion and opinion certainty.

### Example 1:
This example illustrates the impact of the opinion certainty $c_{I,b,n}$ on the final brand ordering. Setting maximum opinion certainty $c_{I,b,n}$ = 1 in the primary modeller's low scores for the Brand_1 brand should lead to the preference for the Brand_2 as the top brand, Brand_3 being the second most preferred brand.

The primary modeller's opinion is configured to prefer the brands in the following order:

**Brand_2 > Brand_3 > Brand_1**.

#### Experiment setup

##### Agent

In [ ]:
# The primary modeller's opinion

# Array representing the scores given by the primary modeller for different phone brands and features
# * Note 1: This array represents primary modellers opinion
primary_modeller_scores = np.array([
    [3, 4, 1],  # Scores for Brand_1
    [6, 5, 6],  # Scores for Brand_2
    [4, 4, 3]   # Scores for Brand_3
], dtype=np.int8)

In [ ]:
# Create a DataFrame
primary_modeller_scores_df = pd.DataFrame(
    primary_modeller_scores,
    index=["Brand 1", "Brand 2", "Brand 3"],
    columns=["Feature 1", "Feature 2", "Feature 3"]
)

# Display the DataFrame
print("The Primary Modellers Scores Table:")
primary_modeller_scores_df

The primary modeller's certainty in the opinion $c_{I,b,n}$ for each brand is the following: 

In [ ]:
# Setting up the opinion certainty

# Array representing the certainty of the primary modeller's opinion for each phone brand
opinion_certainty_array = np.array([1, 0.4, 0.1], dtype=np.float16)

# Boolean flag to determine if certainty should be applied
# * Note 1: This flag is important as setting it to True will ensure that the opinion certainty array is ones.
apply_certainty = True


In [ ]:
# Round the values and convert them to percentage strings
opinion_certainty_percent = [f"{round(value * 100, 1)}%" for value in opinion_certainty_array]

# Create a DataFrame for opinion certainty percentages
opinion_certainty_df = pd.DataFrame(
    [opinion_certainty_percent],             # Percentage values as a list
    columns=["Brand 1", "Brand 2", "Brand 3"]  # Column labels for the DataFrame
)

# Display the DataFrame
print("Opinion Certainty Table:")
opinion_certainty_df


In [ ]:
# Setting up the number of responders for each brand so that the opinion
# certainty can be applied as many times as the responders responded

# Array representing the number of responders for each phone brand
number_of_responders = np.array([10, 10, 10], dtype=np.int32)

In [ ]:
# Create a trust matrix with ones, representing equal trust for each responder
# The matrix has 3 rows (one for each phone brand) and 10 columns (one for each responder)
# * Note 1: Setting trust like this is equivalent to no inclusion of trust
trust_matrix = np.ones((3, 10), dtype=np.float16)

In [ ]:
# Primary modeller's score preference for each feature

# Array representing the primary modeller's score preference for each feature
# * Note 1: This means that scores below the selected scores are not taken into acoount durring the opinion formulation
primary_modeller_score_preference = np.array([4, 4, 4], dtype=np.int8)

In [ ]:
# Primary modeller's brand preference

# Array representing the primary modeller's initial brand preference
# * Note 1: Set to uniform
primary_modeller_brand_pref = np.array([1, 1, 1], dtype=np.float32)

# Normalize the brand preference so that the sum equals 1
primary_modeller_brand_pref = primary_modeller_brand_pref / np.sum(primary_modeller_brand_pref)


In [ ]:
# Setting score range

# This represents the highest possible score for each feature, i.e., score range = 'best'
score_range = 6

In [ ]:
# The values of exponents in Dirichlet distribution must be non-zero,
# setting them to a low value

# Initial feature weight for the Dirichlet distribution
initial_feature_weight = 0.01


##### Expert opinions

Low values of opinion certainty $c_{I,b,n}$ for Brand_2 and Brand_3, result in higher influence of the experts' opinion on the opinion of the primary modeller. This can be intuitively interpreted as the primary modeller being more receptive to advice from the experts $E_{i}$.

In [ ]:
# Simulated experts for Brand_1
# Array representing the opinions of 10 simulated experts on Brand_1 for 3 features
Brand_1_expert_opinions = np.array([
    [4, 3, 4], [5, 3, 5], [5, 6, 5], [6, 5, 3], [6, 6, 6], 
    [5, 6, 5], [6, 6, 5], [6, 3, 4], [4, 5, 4], [6, 4, 3]
], dtype=np.int8)

# Simulated experts for Brand_2
# Array representing the opinions of 10 simulated experts on Brand_2 for 3 features
Brand_2_expert_opinions = np.array([
    [5, 5, 5], [5, 6, 5], [3, 4, 4], [3, 4, 5], [4, 5, 5], 
    [5, 6, 4], [6, 6, 6], [5, 6, 6], [4, 3, 4], [4, 6, 3]
], dtype=np.int8)

# Simulated experts for Brand_3
# Array representing the opinions of 10 simulated experts on Brand_3 for 3 features
Brand_3_expert_opinions = np.array([
    [3, 4, 3], [3, 3, 4], [4, 3, 4], [3, 3, 3], [5, 4, 3], 
    [3, 5, 5], [4, 5, 6], [4, 3, 2], [3, 4, 3], [4, 3, 4]
], dtype=np.int8)

In [ ]:
# Creating DataFrames for each brand

# DataFrame for Brand_1 expert opinions
Brand_1_expert_opinions_df = pd.DataFrame(
    Brand_1_expert_opinions, 
    index=[f"Brand_1 expert {i+1}" for i in range(10)],  # Setting index as expert identifiers
    columns=["Feature 1 score", "Feature 2 score", "Feature 3 score"]  # Setting the column labels for features
)

# DataFrame for Brand_2 expert opinions
Brand_2_expert_opinions_df = pd.DataFrame(
    Brand_2_expert_opinions,
    index=[f"Brand_2 expert {i+1}" for i in range(10)],  # Setting index as expert identifiers
    columns=["Feature 1 score", "Feature 2 score", "Feature 3 score"]  # Setting the column labels for features
)

# DataFrame for Brand_3 expert opinions
Brand_3_expert_opinions_df = pd.DataFrame(
    Brand_3_expert_opinions, 
    index=[f"Brand_3 expert {i+1}" for i in range(10)],  # Setting index as expert identifiers
    columns=["Feature 1 score", "Feature 2 score", "Feature 3 score"]  # Setting the column labels for features
)

<ins>Simple inference of experts opinion of brands</ins>

In [ ]:
# Calculate the mean scores of each expert
Brand_1_means = Brand_1_expert_opinions_df.mean(axis=1)
Brand_2_means = Brand_2_expert_opinions_df.mean(axis=1)
Brand_3_means = Brand_3_expert_opinions_df.mean(axis=1)

# Create a box plot for each brand
fig, ax = plt.subplots(figsize=(10, 6))

ax.boxplot([Brand_1_means, Brand_2_means, Brand_3_means], labels=["Brand 1", "Brand 2", "Brand 3"], showmeans=True)
ax.set_title('Box Plot of Mean Expert Scores for Each Brand', fontsize=15.5)
ax.set_ylabel('Mean Score')
ax.set_xlabel('Brand')

# Adding scatter plot of the observations
x_positions = [1, 2, 3]
for i, means in enumerate([Brand_1_means, Brand_2_means, Brand_3_means]):
    ax.scatter([x_positions[i]] * len(means), means, alpha=0.6, color='blue', s=5)

# Save the figure in .eps format

# save_path = os.path.join(working_directory, 'Bar_charts', 'brand_mean_scores_boxplot.eps')
# plt.tight_layout()
# plt.savefig(save_path, format='eps')

# Show the plot
plt.show()


In [ ]:
# Running the simulated example from the auxiliary module

# Call the simulated_example function with the provided parameters
_, primary_modeller_posterior_updated = auxiliary.simulated_example(
    primary_modeller_scores,  # Initial scores given by the primary modeller
    opinion_certainty_array,  # Certainty of the primary modeller's opinions
    apply_certainty,  # Flag to apply certainty
    number_of_responders,  # Number of responders for each brand
    trust_matrix,  # Trust matrix representing trust for each responder
    primary_modeller_score_preference,  # Primary modeller's score preference for each feature
    primary_modeller_brand_pref,  # Distribution representing primary modeller's brand prefference
    score_range,  # Highest possible score for each feature
    initial_feature_weight,  # Initial feature weight for the Dirichlet distribution
    Brand_1_expert_opinions,  # Expert opinions for Brand_1
    Brand_2_expert_opinions,  # Expert opinions for Brand_2
    Brand_3_expert_opinions  # Expert opinions for Brand_3
)


In [ ]:
## Plot primary modeller's initial and updated posterior distribution on brands

# Create a figure window with two subplots
fig, axes = plt.subplots(1, 1, figsize=(7, 5))

# Variable names for the x-ticks
variable_names = ['Brand 1', 'Brand 2', 'Brand 3']

# Plotting the initial posterior distribution as a bar chart
axes.bar(variable_names, primary_modeller_posterior_updated)
axes.set_ylabel('Probability')
axes.set_title("Primary modeller's updated brand preference", fontsize=15.5)
axes.tick_params(axis='x', labelsize=17)

# Save the figure
save_path = os.path.join(working_directory, 'Bar_charts', 'figure_5.eps')
plt.tight_layout()
plt.savefig(save_path, format='eps')

# Show the plots
plt.show()




<ins>The primary modeller's updated brand preference.</ins>

In [ ]:
# Round the values and convert them to percentage strings
primary_modeller_posterior_updated_percent = [f"{value * 100:.2f}%" for value in primary_modeller_posterior_updated]

# Create a DataFrame for the updated posterior percentages
primary_modeller_posterior_updated_df = pd.DataFrame(
    [primary_modeller_posterior_updated_percent],    # Percentage values as a list
    columns=["Brand 1", "Brand 2", "Brand 3"]         # Column labels for the DataFrame
)

# Display the DataFrame
print("The Primary modeller's updated brand preference:")
primary_modeller_posterior_updated_df

The opinion certainty process works as intended in this example.

### Example 2
In this example the maximum opinion certainty $c_{I,b,n}$ = 1 in the primary modeller's low scores for the Brand_1 and Brand_2 brand should lead to the preference for the Brand_3 brand.

The primary modeller's opinion is configured to prefer the brands in the following order:

**Brand_3 > Brand_2 > Brand_1**. 

#### Experiment setup

##### Agent

In [ ]:
# The primary modeller's opinion

# Array representing the scores given by the primary modeller for different phone brands and features
# * Note 1: This array represents primary modellers opinion
primary_modeller_scores = np.array([
    [3, 1, 1],  # Scores for Brand_1
    [2, 3, 1],  # Scores for Brand_2
    [6, 5, 5]   # Scores for Brand_3
], dtype=np.int8)


In [ ]:
# Creating a DataFrame with the given data
# Index represents phone brands, and columns represent different features
primary_modeller_scores_df = pd.DataFrame(
    primary_modeller_scores,
    index=["Brand 1", "Brand 2", "Brand 3"],           # Setting the row labels for phone brands
    columns=["Feature 1", "Feature 2", "Feature 3"]  # Setting the column labels for features
)

# Display the DataFrame
primary_modeller_scores_df

The primary modeller's certainty in the opinion $c_{I,b,n}$ for each brand is the following: 

In [ ]:
# Setting up the opinion certainty

# Array representing the certainty of the primary modeller's opinion for each phone brand
opinion_certainty_array = np.array([0.2, 0.2, 1], dtype=np.float16)

# Boolean flag to determine if certainty should be applied
# * Note 1: This flag is important as setting it to True will ensure that the opinion certainty array is ones.
apply_certainty = True

In [ ]:
# Round the values and convert them to percentage strings
opinion_certainty_percent = [f"{round(value * 100, 1)}%" for value in opinion_certainty_array]

# Create a DataFrame for opinion certainty percentages
opinion_certainty_df = pd.DataFrame(
    [opinion_certainty_percent],             # Percentage values as a list
    columns=["Brand 1", "Brand 2", "Brand 3"]  # Column labels for the DataFrame
)

# Display the DataFrame
print("Opinion Certainty Table:")
opinion_certainty_df


Similarly to the previous example, low values of opinion certainty $c_{I,b,n}$ for Brand_1 and Brand_2, result in higher influence of the experts' opinion on the opinion of the primary modeller. This can be intuitively interpreted as the primary modeller being more receptive to advice from the experts $E_{i}$.

In [ ]:
# Setting up the number of responders for each brand so that the opinion
# certainty can be applied as many times as the responders responded

# Array representing the number of responders for each phone brand
number_of_responders = np.array([10, 10, 10], dtype=np.int32)

In [ ]:
# Create a trust matrix with ones, representing equal trust for each responder
# The matrix has 3 rows (one for each phone brand) and 10 columns (one for each responder)
# * Note 1: Setting trust like this is equivalent to no inclusion of trust
trust_matrix = np.ones((3, 10), dtype=np.float16)

In [ ]:
# Primary modeller's score preference for each feature

# Array representing the primary modeller's score preference for each feature
# * Note 1: This means that scores below the selected scores are not taken into acoount durring the opinion formulation
primary_modeller_score_preference = np.array([4, 4, 4], dtype=np.int8)

In [ ]:
# Primary modeller's brand preference

# Array representing the primary modeller's initial brand preference
# * Note 1: Set to uniform
primary_modeller_brand_pref = np.array([1, 1, 1], dtype=np.float32)

# Normalize the brand preference so that the sum equals 1
primary_modeller_brand_pref = primary_modeller_brand_pref / np.sum(primary_modeller_brand_pref)


In [ ]:
# Setting score range

# This represents the highest possible score for each feature, i.e., score range = 'best'
score_range = 6

In [ ]:
# The values of exponents in Dirichlet distribution must be non-zero,
# setting them to a low value

# Initial feature weight for the Dirichlet distribution
initial_feature_weight = 0.01


##### Expert opinions

In [ ]:
# Simulated experts for Brand_1
# Array representing the opinions of 10 simulated experts on Brand_1 for 3 features
Brand_1_expert_opinions = np.array([
    [4, 3, 4], [5, 3, 5], [5, 6, 5], [6, 5, 3], [6, 6, 6], 
    [5, 6, 5], [6, 6, 5], [6, 3, 4], [4, 5, 4], [6, 4, 3]
], dtype=np.int8)

# Simulated experts for Brand_2
# Array representing the opinions of 10 simulated experts on Brand_2 for 3 features
Brand_2_expert_opinions = np.array([
    [5, 5, 5], [5, 6, 5], [3, 4, 4], [3, 4, 5], [4, 5, 5], 
    [5, 6, 4], [6, 6, 6], [5, 6, 6], [4, 3, 4], [4, 6, 3]
], dtype=np.int8)

# Simulated experts for Brand_3
# Array representing the opinions of 10 simulated experts on Brand_3 for 3 features
Brand_3_expert_opinions = np.array([
    [3, 4, 3], [3, 3, 4], [4, 3, 4], [3, 3, 3], [5, 4, 3], 
    [3, 5, 5], [4, 5, 6], [4, 3, 2], [3, 4, 3], [4, 3, 4]
], dtype=np.int8)


In [ ]:
# Creating DataFrames for each brand

# DataFrame for Brand_1 expert opinions
Brand_1_expert_opinions_df = pd.DataFrame(
    Brand_1_expert_opinions, 
    index=[f"Brand_1 expert {i+1}" for i in range(10)],  # Setting index as expert identifiers
    columns=["Feature 1 score", "Feature 2 score", "Feature 3 score"]  # Setting the column labels for features
)

# DataFrame for Brand_2 expert opinions
Brand_2_expert_opinions_df = pd.DataFrame(
    Brand_2_expert_opinions,
    index=[f"Brand_2 expert {i+1}" for i in range(10)],  # Setting index as expert identifiers
    columns=["Feature 1 score", "Feature 2 score", "Feature 3 score"]  # Setting the column labels for features
)

# DataFrame for Brand_3 expert opinions
Brand_3_expert_opinions_df = pd.DataFrame(
    Brand_3_expert_opinions, 
    index=[f"Brand_3 expert {i+1}" for i in range(10)],  # Setting index as expert identifiers
    columns=["Feature 1 score", "Feature 2 score", "Feature 3 score"]  # Setting the column labels for features
)

<ins>Simple inference of experts opinion of brands</ins>

In [ ]:
# Calculate the mean scores of each expert
Brand_1_means = Brand_1_expert_opinions_df.mean(axis=1)
Brand_2_means = Brand_2_expert_opinions_df.mean(axis=1)
Brand_3_means = Brand_3_expert_opinions_df.mean(axis=1)

# Create a box plot for each brand
fig, ax = plt.subplots(figsize=(10, 6))

ax.boxplot([Brand_1_means, Brand_2_means, Brand_3_means], labels=["Brand 1", "Brand 2", "Brand 3"], showmeans=True)
ax.set_title('Box Plot of Mean Expert Scores for Each Brand', fontsize=15.5)
ax.set_ylabel('Mean Score')
ax.set_xlabel('Brand')

# Adding scatter plot of the observations
x_positions = [1, 2, 3]
for i, means in enumerate([Brand_1_means, Brand_2_means, Brand_3_means]):
    ax.scatter([x_positions[i]] * len(means), means, alpha=0.6, color='blue', s=5)

# Save the figure in .eps format

# save_path = os.path.join(working_directory, 'Bar_charts', 'brand_mean_scores_boxplot.eps')
# plt.tight_layout()
# plt.savefig(save_path, format='eps')

# Show the plot
plt.show()


In [ ]:
# Running the simulated example from the auxiliary module

# Call the simulated_example function with the provided parameters
_, primary_modeller_posterior_updated = auxiliary.simulated_example(
    primary_modeller_scores,  # Initial scores given by the primary modeller
    opinion_certainty_array,  # Certainty of the primary modeller's opinions
    apply_certainty,  # Flag to apply certainty
    number_of_responders,  # Number of responders for each brand
    trust_matrix,  # Trust matrix representing trust for each responder
    primary_modeller_score_preference,  # Primary modeller's score preference for each feature
    primary_modeller_brand_pref,  # Distribution representing primary modeller's brand prefference
    score_range,  # Highest possible score for each feature
    initial_feature_weight,  # Initial feature weight for the Dirichlet distribution
    Brand_1_expert_opinions,  # Expert opinions for Brand_1
    Brand_2_expert_opinions,  # Expert opinions for Brand_2
    Brand_3_expert_opinions  # Expert opinions for Brand_3
)


In [ ]:
## Plot primary modeller's initial and updated posterior distribution on brands

# Create a figure window with two subplots
fig, axes = plt.subplots(1, 1, figsize=(7, 5))

# Variable names for the x-ticks
variable_names = ['Brand 1', 'Brand 2', 'Brand 3']

# Plotting the updated posterior distribution as a bar chart
axes.bar(variable_names, primary_modeller_posterior_updated)
axes.set_ylabel('Probability')
axes.set_title("Primary modeller's updated brand preference", fontsize=15.5)
axes.tick_params(axis='x', labelsize=17)

# Save the figure
save_path = os.path.join(working_directory, 'Bar_charts', 'figure_6.eps')
plt.tight_layout()
plt.savefig(save_path, format='eps')

# Show the plots
plt.show()


<ins>The primary modeller's updated brand preference.</ins>

In [ ]:
# Round the values and convert them to percentage strings
primary_modeller_posterior_updated_percent = [f"{value * 100:.2f}%" for value in primary_modeller_posterior_updated]

# Create a DataFrame for the updated posterior percentages
primary_modeller_posterior_updated_df = pd.DataFrame(
    [primary_modeller_posterior_updated_percent],    # Percentage values as a list
    columns=["Brand 1", "Brand 2", "Brand 3"]         # Column labels for the DataFrame
)

# Display the DataFrame
print("The Primary modeller's updated brand preference:")
primary_modeller_posterior_updated_df

The opinion certainty process works as intended in this example.

### Example 3
In this example the maximum opinion certainty $c_{I,b,n}$ = 1 in the primary modeller's lowest scores for Brand_1 brand should lead to the preference for the Brand_2 brand.

The primary modeller's opinion is configured to prefer the brands in the following order:

**Brand_2 > Brand_3 > Brand_1**. 

#### Experiment setup

##### Agent

In [ ]:
# The primary modeller's opinion

# Array representing the scores given by the primary modeller for different phone brands and features
# * Note 1: This array represents primary modellers opinion
primary_modeller_scores = np.array([
    [1, 1, 1],  # Scores for Brand_1
    [6, 5, 6],  # Scores for Brand_2
    [4, 4, 3]   # Scores for Brand_3
], dtype=np.int8)


In [ ]:
# Creating a DataFrame with the given data
# Index represents phone brands, and columns represent different features
primary_modeller_scores_df = pd.DataFrame(
    primary_modeller_scores,
    index=["Brand 1", "Brand 2", "Brand 3"],           # Setting the row labels for phone brands
    columns=["Feature 1", "Feature 2", "Feature 3"]  # Setting the column labels for features
)

# Display the DataFrame
primary_modeller_scores_df

Additionally, to simulate an issue with this design, all the experts' opinions for Brand_1 are deliberately set to the highest scores.

The primary modeller's certainty in the opinion $c_{I,b,n}$ for each brand is the following: 

In [ ]:
# Setting up the opinion certainty

# Array representing the certainty of the primary modeller's opinion for each phone brand
opinion_certainty_array = np.array([1, 0, 0], dtype=np.float16)

# Boolean flag to determine if certainty should be applied
# * Note 1: This flag is important as setting it to True will ensure that the opinion certainty array is ones.
apply_certainty = True

In [ ]:
# Round the values and convert them to percentage strings
opinion_certainty_percent = [f"{round(value * 100, 1)}%" for value in opinion_certainty_array]

# Create a DataFrame for opinion certainty percentages
opinion_certainty_df = pd.DataFrame(
    [opinion_certainty_percent],             # Percentage values as a list
    columns=["Brand 1", "Brand 2", "Brand 3"]  # Column labels for the DataFrame
)

# Display the DataFrame
print("Opinion Certainty Table:")
opinion_certainty_df

In [ ]:
# Setting up the number of responders for each brand so that the opinion
# certainty can be applied as many times as the responders responded

# Array representing the number of responders for each phone brand
number_of_responders = np.array([10, 10, 10], dtype=np.int32)

In [ ]:
# Create a trust matrix with ones, representing equal trust for each responder
# The matrix has 3 rows (one for each phone brand) and 10 columns (one for each responder)
# * Note 1: Setting trust like this is equivalent to no inclusion of trust
trust_matrix = np.ones((3, 10), dtype=np.float16)

In [ ]:
# Primary modeller's score preference for each feature

# Array representing the primary modeller's score preference for each feature
# * Note 1: This means that scores below the selected scores are not taken into acoount durring the opinion formulation
primary_modeller_score_preference = np.array([4, 4, 4], dtype=np.int8)

In [ ]:
# Primary modeller's brand preference

# Array representing the primary modeller's initial brand preference
# * Note 1: Set to uniform
primary_modeller_brand_pref = np.array([1, 1, 1], dtype=np.float16)

# Normalize the brand preference so that the sum equals 1
primary_modeller_brand_pref = primary_modeller_brand_pref / np.sum(primary_modeller_brand_pref)


In [ ]:
# Setting score range

# This represents the highest possible score for each feature, i.e., score range = 'best'
score_range = 6

In [ ]:
# The values of exponents in Dirichlet distribution must be non-zero,
# setting them to a low value

# Initial feature weight for the Dirichlet distribution
initial_feature_weight = 0.01


When opinion certainty values $c_{I,b,n}$ for Brand_2 and Brand_3 are set to zero, it results in the complete influence of the experts' opinions on the primary modeller's own opinion. This can be intuitively interpreted as the primary modeller completely replacing his own opinion with the opinions of the experts $E_{i}$.

##### Expert opinions

In [ ]:
# Simulated experts for Brand_1
# Array representing the opinions of 10 simulated experts on Brand_1 for 3 features
Brand_1_expert_opinions = np.array([
    [6, 6, 6], [6, 6, 6], [6, 6, 6], [6, 6, 6], [6, 6, 6], 
    [6, 6, 6], [6, 6, 6], [6, 6, 6], [6, 6, 6], [6, 6, 6]
], dtype=np.int8)

# Simulated experts for Brand_2
# Array representing the opinions of 10 simulated experts on Brand_2 for 3 features
Brand_2_expert_opinions = np.array([
    [5, 5, 5], [5, 6, 5], [3, 4, 4], [3, 4, 5], [4, 5, 5], 
    [5, 6, 4], [6, 6, 6], [5, 6, 6], [4, 3, 4], [4, 6, 3]
], dtype=np.int8)

# Simulated experts for Brand_3
# Array representing the opinions of 10 simulated experts on Brand_3 for 3 features
Brand_3_expert_opinions = np.array([
    [3, 4, 3], [3, 3, 4], [4, 3, 4], [3, 3, 3], [5, 4, 3], 
    [3, 5, 5], [4, 5, 6], [4, 3, 2], [3, 4, 3], [4, 3, 4]
], dtype=np.int8)

In [ ]:
# Creating DataFrames for each brand

# DataFrame for Brand_1 expert opinions
Brand_1_expert_opinions_df = pd.DataFrame(
    Brand_1_expert_opinions, 
    index=[f"Brand_1 expert {i+1}" for i in range(10)],  # Setting index as expert identifiers
    columns=["Feature 1 score", "Feature 2 score", "Feature 3 score"]  # Setting the column labels for features
)

# DataFrame for Brand_2 expert opinions
Brand_2_expert_opinions_df = pd.DataFrame(
    Brand_2_expert_opinions,
    index=[f"Brand_2 expert {i+1}" for i in range(10)],  # Setting index as expert identifiers
    columns=["Feature 1 score", "Feature 2 score", "Feature 3 score"]  # Setting the column labels for features
)

# DataFrame for Brand_3 expert opinions
Brand_3_expert_opinions_df = pd.DataFrame(
    Brand_3_expert_opinions, 
    index=[f"Brand_3 expert {i+1}" for i in range(10)],  # Setting index as expert identifiers
    columns=["Feature 1 score", "Feature 2 score", "Feature 3 score"]  # Setting the column labels for features
)

<ins>Simple inference of experts opinion of brands</ins>

In [ ]:
# Calculate the mean scores of each expert
Brand_1_means = Brand_1_expert_opinions_df.mean(axis=1)
Brand_2_means = Brand_2_expert_opinions_df.mean(axis=1)
Brand_3_means = Brand_3_expert_opinions_df.mean(axis=1)

# Create a box plot for each brand
fig, ax = plt.subplots(figsize=(10, 6))

ax.boxplot([Brand_1_means, Brand_2_means, Brand_3_means], labels=["Brand 1", "Brand 2", "Brand 3"], showmeans=True)
ax.set_title('Box Plot of Mean Expert Scores for Each Brand', fontsize=15.5)
ax.set_ylabel('Mean Score')
ax.set_xlabel('Brand')

# Adding scatter plot of the observations
x_positions = [1, 2, 3]
for i, means in enumerate([Brand_1_means, Brand_2_means, Brand_3_means]):
    ax.scatter([x_positions[i]] * len(means), means, alpha=0.6, color='blue', s=5)

# Save the figure in .eps format

# save_path = os.path.join(working_directory, 'Bar_charts', 'brand_mean_scores_boxplot.eps')
# plt.tight_layout()
# plt.savefig(save_path, format='eps')

# Show the plot
plt.show()


In [ ]:
# Running the simulated example from the auxiliary module

# Call the simulated_example function with the provided parameters
_, primary_modeller_posterior_updated = auxiliary.simulated_example(
    primary_modeller_scores,  # Initial scores given by the primary modeller
    opinion_certainty_array,  # Certainty of the primary modeller's opinions
    apply_certainty,  # Flag to apply certainty
    number_of_responders,  # Number of responders for each brand
    trust_matrix,  # Trust matrix representing trust for each responder
    primary_modeller_score_preference,  # Primary modeller's score preference for each feature
    primary_modeller_brand_pref,  # Distribution representing primary modeller's brand prefference
    score_range,  # Highest possible score for each feature
    initial_feature_weight,  # Initial feature weight for the Dirichlet distribution
    Brand_1_expert_opinions,  # Expert opinions for Brand_1
    Brand_2_expert_opinions,  # Expert opinions for Brand_2
    Brand_3_expert_opinions  # Expert opinions for Brand_3
)


In [ ]:
## Plot primary modeller's initial and updated posterior distribution on brands

# Create a figure window with two subplots
fig, axes = plt.subplots(1, 1, figsize=(7, 5))

# Variable names for the x-ticks
variable_names = ['Brand 1', 'Brand 2', 'Brand 3']

# Plotting the updated posterior distribution as a bar chart
axes.bar(variable_names, primary_modeller_posterior_updated)
axes.set_ylabel('Probability')
axes.set_title("Primary modeller's updated brand preference", fontsize=15.5)
axes.tick_params(axis='x', labelsize=17)

# Save the figure
save_path = os.path.join(working_directory, 'Bar_charts', 'figure_7.eps')
plt.tight_layout()
plt.savefig(save_path, format='eps')

# Show the plots
plt.show()


<ins>The primary modeller's updated brand preference.</ins>

In [ ]:
# Round the values and convert them to percentage strings
primary_modeller_posterior_updated_percent = [f"{value * 100:.2f}%" for value in primary_modeller_posterior_updated]

# Create a DataFrame for the updated posterior percentages
primary_modeller_posterior_updated_df = pd.DataFrame(
    [primary_modeller_posterior_updated_percent],    # Percentage values as a list
    columns=["Brand 1", "Brand 2", "Brand 3"]         # Column labels for the DataFrame
)

# Display the DataFrame
print("The Primary modeller's updated brand preference:")
primary_modeller_posterior_updated_df

In this scenario, an unexpected outcome occurs due to the way each expert's $E_{i}$ contribution to the updated weight $V_{f_{j,b}}$ and the primary modeller's contribution are configured. In particular:

 1. Each expert's contribution to the updated weight $V_{f_{i,b}}$ has a magnitude of +1.
 2. The primary modeller's contribution to the updated weight $V_{f_{i,b}}$ is also +1 for each expert, given that opinion certainty $c_{I,b,n}$ is set to the maximum value of 1.

As a result, the updated weight becomes $V_{i_{Brand_1}}$ = 10 for all $i\in \{1,2,3\}$. This leads to high maximum values for the respective probabilities $P_{I}^{0}(F_{i,Brand_1}|Brand_1)$ for all $i\in \{1,2,3\}$, of choosing the underlying scores. Consequently, the probabilities entering the final posterior distribution on brands $P_{I}^{0}(F_{1,Brand_1}|Brand_1)P_{I}^{0}(F_{2,Brand_1}|Brand_1)P_{I}^{0}(F_{3,Brand_1}|Brand_1)$ remain high, resulting in the selection of the Brand_1 brand, contrary to the expected outcome.